In [297]:


class Parser(object):
    '''
    Get the number of layers
    Get the number of timesteps
    Get the name of edits #the paramters
    
    Get the layer-dependent data
    Get the spectral data
    '''
    def __init__(self,filename,spectrum_keys=['SPECTRUM3'],save_list=['SPECTRUM3','r vs tiv']):
        self.filename=filename
        
        self.list_of_spectrum_keys=spectrum_keys
        %time self.aggregate_dictionary()
        self.save_list=save_list
        self.list_of_datablocks=[]
        
    def parse_file(self):
        '''Placeholder to actually go over the file and save the data'''

        with open(self.filename) as f:
            for line_index,line in enumerate(f):
                line=line.rstrip()
                if line.startswith('#'):
                    time=line.split()[3]
                    edit=str.join(' ', line.split()[5:])
                    self.list_of_datablocks.append(DataBlock(line_index,time,edit))
                elif line.startswith('$'):
                    self.list_of_datablocks[-1].add_descriptor_line(line)
                else:
                    self.list_of_datablocks[-1].add_line(line)
                    
                            
#        with open(self.filename) as f:
#            for line_index,line in enumerate(f):
#                if parse_status=='search':
#                    for save_edit in save_list:
#                        if self.save_list in line:
#                            print line
#                            if search_edit in self.save_list:
#                                parse_status='saving'
#                            else: 
#                                parse_status='skipping'
                     
        #print 'We are parsing {0}, looking for "{1}" in {2} \n'.format(self.filename,key_to_look_for,layers_to_use)
        

                
    def stupid_output(self):
        '''A stupid function that simply print the original file line by line'''
        with open(self.filename) as f:
            for line in f:
                print line
                    
    def aggregate_dictionary(self): 
        '''Reads the file to find the string that represent time'''

        full_list_of_edits=[]
        full_list_of_times=[]
        self.timesteps_str=[]
        self.edit_name_list=[]
        
        line_index_before=0
        line_diff=0
        line_diff_list=[]
        
    
        with open(self.filename) as f:
            for line_index,line in enumerate(f):
                if 'time' in line:
                    full_list_of_times.append(line.split()[3])
                    full_list_of_edits.append(str.join(' ', line.split()[5:]))

        for i in full_list_of_times:
            if i not in self.timesteps_str:
                self.timesteps_str.append(i)
                
        self.timesteps_float=[float(temp) for temp in self.timesteps_str]
                    
        for i in full_list_of_edits:
            if i not in self.edit_name_list:
                self.edit_name_list.append(i)
        
        with open(self.filename) as f:
            for line_index,line in enumerate(f):
                if 'time' in line:
                    #print '{0} found at line {1}'.format(line.split(),line_index)
                    full_list_of_times.append(line.split()[3])
                    full_list_of_edits.append(str.join(' ', line.split()[5:]))
                    for edit_index,edit in enumerate(self.edit_name_list):
                        if edit in line:
                            line_index_diff=line_index-line_index_before
                            line_index_before=line_index
                            line_diff_list.append(line_index_diff)
        
        self.edit_length_list=line_diff_list[1:len(self.edit_name_list)+1]
                
        self.edit_dict_list=[]
        
        for edit_index,edit_name in enumerate(self.edit_name_list):
            if self.edit_name_list[edit_index] in self.list_of_spectrum_keys:
                edit_type='spectral'
            else:
                edit_type='layered'
                self.number_of_layers=self.edit_length_list[edit_index]-3
            edit={'Name':self.edit_name_list[edit_index],'Length':self.edit_length_list[edit_index],'Type':edit_type,'Order':edit_index}
            self.edit_dict_list.append(edit)

            
class DataBlock(object):
    '''Datablock for text'''
    def __init__(self,lineindex,time,edit):
        self.lineindex=lineindex
        self.time=time
        self.edit=edit
        self.descriptor_lines=[]
        self.lines=[]
    def add_descriptor_line(self,line):
        '''Add a line that contain column descriptors'''
        self.descriptor_lines.append([line])
    def add_line(self,line):
        '''Add a line that contain actual data'''
        self.number_of_columns=len(line.split())
        self.lines.append([line])
    def cleanup(self):
        self.column_descriptions1=self.descriptor_lines[0][0].split()[1:]
        #if 'r' in description:
        #    self.data_type='layered'
        

        for description_index,description in enumerate(self.column_descriptions1):
            if 
            self.column_descriptions2=my_parse.list_of_datablocks[0].descriptor_lines[1][0].split()[1:]


my_parse=Parser('run.plt')
#my_list=my_pars.timesteps_str
#print my_list
my_list=my_parse.edit_dict_list
#print my_list
#print my_pars.number_of_layers

my_parse.parse_file()
for i in my_parse.list_of_datablocks:
    i.cleanup()
print my_parse.list_of_datablocks[0].column_descriptions2
#[i['Name'] for i in my_list]

#print my_parse.list_of_datablocks[0].lines[0][0].split()
#my_parse.list_of_datablocks[0].clean_up()

SyntaxError: invalid syntax (<ipython-input-297-4c390c34364a>, line 133)

In [193]:
            
class Timestep(object):
    '''Describes one timestep'''
    def __init__(self,time,timeindex,**spectrum):
        self.time=time
        self.index=timeindex
        self.list_of_layers=layers
        self.spectrum=Spectrum(**spectrum)

class Spectrum(object):
    '''Describes one spectrum'''
    def __init__(self,energybins,intensity):
        self.energybins=energybins #a list
        self.intensity=intensity # a list
        self.energy_range=[min(energybins),max(energybins)]
        self.intensity_range=[min(intensity),max(intensity)]
        self._clean_spectrum = None

        
        



class Experiment(object):
    #list of simulations
    
    #method: create plots given a range of energies
    
    '''
    
    '''
    pass

class Simulation(object):
    def __init__(self):
        # Do parsing
        # create a list of timesteps
        #all the sim parameters
        pass
    def subtract():
        pass
    
class Layer(object):
    '''Describes one layer'''
    def __init__(self,name,composition,average_ionization,energybins,intensity,layer):
        self.name=name
        self.composition=composition
        self.position_index=position_index
        #self.ionization=average_ionization
        #self.temperature=temperature
        self.timestep_it_belongs_to=timestep
        self.Spectrum=Spectrum(energybins,intensity)

class Timestep(object):
    '''Describes one timestep'''
    def __init__(self,time,timeindex):
        self.time=time
        self.line_index=line_index
        self.list_of_layers=layers
        
class Spectrum(object):
    '''Describes one spectrum'''
    def __init__(self,energybins,intensity):
        self.energybins=energybins #a list
        self.intensity=intensity # a list
        self.energy_range=[min(energybins),max(energybins)]
        self.intensity_range=[min(intensity),max(intensity)]
        self._clean_spectrum = None
        
    def return_spectrum(self):
        return self.energybins,self.energy_range
    
    def clean_spectrum(self):
        '''
        EXPENSIVE
        '''            
        if self._clean_spectrum is None:
            r = threshold
            for i in range(100000000000):
                r += i
            self._clean_spectrum = r
        else:
            return self._clean_spectrum
        
    def wash_spectrum(self):
        '''
        EXPENSIVE
        '''
        if not hasattr(self, "_wash_spectrum"):
            r = sum(range(10000000))
            self._wash_spectrum = r
        return self._wash_spectrum
        
    def calculate_spectrum(self,background_spectrum=None,method='subtract'):
        self.background_spectrum=background_spectrum
        if method==subtract:
            self.spectrum=self.spectrum-self.background_spectrum
            return
        else:
            print 'No background correction done'
            return

class SpectrumPlot(object):
    '''
    A plot that describes spectral data
    '''
    def __init__(self,spectrum,ranges):
        self.spectrum=spectrum
        self.ranges=ranges
    def create_heat_panel(self):
        pass
    def create_sum_panel(self):
        pass
    def show_composite_plot(self):
        pass            

In [ ]:
pwd

In [78]:
#['r', 'vs', 'number_of_i']

In [79]:
x=None
assert x

AssertionError: 

In [175]:
x={'Name':'Ola','Species':'apa'}
print x
x.update({'Age':5})
print x

{'Name': 'Ola', 'Species': 'apa'}
{'Age': 5, 'Name': 'Ola', 'Species': 'apa'}
